In [ ]:
## Notebook env: brian_script_env (R kernel)
## this notebook finds variable genes necessary for miR_heterogeneity_nb.ipynb and Data_prepping.ipynb

In [1]:
library(Seurat)
library(SeuratDisk)
library(Matrix)
library(ggplot2)

Warning message:
“package ‘SeuratDisk’ was built under R version 4.0.3”
Registered S3 methods overwritten by 'SeuratObject':
  method                     from  
  $.JackStrawData            Seurat
  $.Seurat                   Seurat
  $.SeuratCommand            Seurat
  $<-.Seurat                 Seurat
  [.Assay                    Seurat
  [.DimReduc                 Seurat
  [.Seurat                   Seurat
  [.SeuratCommand            Seurat
  [.SpatialImage             Seurat
  [[.Assay                   Seurat
  [[.DimReduc                Seurat
  [[.Seurat                  Seurat
  levels<-.Seurat            Seurat
  .DollarNames.JackStrawData Seurat
  .DollarNames.Seurat        Seurat
  .DollarNames.SeuratCommand Seurat
  as.list.SeuratCommand      Seurat
  as.logical.JackStrawData   Seurat
  dim.Assay                  Seurat
  dim.DimReduc               Seurat
  dim.Neighbor               Seurat
  dim.Seurat                 Seurat
  dim.SpatialImage           Seurat
  dimnames.

In [ ]:
raw_meta <- read.csv('/home/ssobti/projects/mir_tud/010523_filtered_data/miR_raw_meta.csv', header = TRUE)
head(raw_meta)
raw_genes <- read.csv('/home/ssobti/projects/mir_tud/010523_filtered_data/miR_raw_genes.csv', header = FALSE)
raw_genes <- raw_genes$V1
RNA_Seq_expression_raw_data <- readMM('/home/ssobti/projects/mir_tud/010523_filtered_data/miR_raw.mtx')
dim(RNA_Seq_expression_raw_data)
rownames(RNA_Seq_expression_raw_data) <- raw_meta$X
colnames(RNA_Seq_expression_raw_data) <- raw_genes
RNA_Seq_expression_raw_data <- Matrix::t(RNA_Seq_expression_raw_data)
miR_seurat_data <- CreateSeuratObject(RNA_Seq_expression_raw_data, min.cells = 3, min.features = 200)

In [ ]:
miR_seurat_data[["percent.mt"]] <- PercentageFeatureSet(miR_seurat_data, pattern = "^MT-")

In [ ]:
miR_seurat_data <- subset(miR_seurat_data, subset = percent.mt < 12)

In [ ]:
miR_seurat_data <- NormalizeData(miR_seurat_data, normalization.method = "LogNormalize", scale.factor = 10000)

In [ ]:
miR_seurat_data <- FindVariableFeatures(miR_seurat_data, selection.method = "vst", nfeatures = 2000)

In [ ]:
VariableFeaturePlot(miR_seurat_data) -> plot1
top10 <- head(VariableFeatures(miR_seurat_data), 10)
plot2 <- LabelPoints(plot = plot1, points = top10, repel = TRUE)
plot2

In [ ]:
top_variable_genes = VariableFeatures(miR_seurat_data)

In [ ]:
top_var_genes = data.frame(variable_genes = top_variable_genes)

In [ ]:
write.csv(top_var_genes, file = '/home/ssobti/projects/mir_tud/output_data/heterogeneity/top_var_genes.csv')

In [ ]:
RNA_Seq_expression_raw_data <- as(RNA_Seq_expression_raw_data, 'dgCMatrix')

In [ ]:
medians = sparseMatrixStats::rowMedians(RNA_Seq_expression_raw_data)

In [ ]:
median_df = data.frame(gene_medians = medians)

In [ ]:
ggplot(median_df, aes(x=gene_medians)) + 
  geom_histogram(color="black", fill="white", bins = 50000) +
  theme_bw()

In [ ]:
ggplot(median_df, aes(x=gene_medians)) + 
  geom_histogram(color="black", fill="white", bins = 50000) +
  coord_cartesian(xlim = c(0,200)) +
  theme_bw()

In [ ]:
ggplot(median_df, aes(x=gene_medians)) + 
  geom_histogram(color="black", fill="white", bins = 50000) +
  coord_cartesian(xlim = c(0,10)) +
  theme_bw()